In [2]:
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import numpy as np
import math
import geopandas as gpd
import fiona

import os
from rasterio.merge import merge, copy_count
from rasterio.mask import mask


Step-by-Step: Download LIDAR DTM GeoTIFF
Go to the official dataset page:
👉 LIDAR Composite DTM - 1m (Environment Agency) 1

Scroll to “Download the survey data” section.

https://environment.data.gov.uk/survey 

Choose your area of interest:

You can either:
Use the interactive map to select tiles, or
Download by OS National Grid tile name (e.g., SJ84 for Newcastle-under-Lyme).
Download the GeoTIFF file:

Each tile is a 5km x 5km raster file in .tif format.
The data is in metres, referenced to OS Newlyn using OSTN’15.

In [ ]:
#DTM =gpd.read_file('/Users/fran/Downloads/LIDAR_DTM_Time_Stamped_Extents.json')

https://spatial-dev.guru/2023/12/10/line-of-sight-analysis-in-digital-elevation-models-using-python/

In [ ]:
# # Replace this with the path to your DEM file
# # Load the DEM data
# dem_data = rasterio.open('/Users/fran/Downloads/lidar_composite_dtm-2022-1-SE02ne/SE02ne_DTM_1m.tif')
# dem_array = dem_data.read(1)
# transform = dem_data.transform

In [ ]:
 
# # Plot DEM data
# plt.imshow(dem_array, cmap='gray')
# plt.colorbar()
# plt.title('Digital Elevation Model')
# plt.show()

In [ ]:
# read in osm highways data
osm= gpd.read_file('/Users/fran/Downloads/OSM_highway.geojson')

In [ ]:
osm.plot()

https://osdatahub.os.uk/downloads/open/OpenGreenspace

In [4]:
# read in open greenspace data (Ordanace Survey)
# check available layers
# layers = fiona.listlayers('/Users/fran/Downloads/opgrsp_gpkg_gb/Data/opgrsp_gb.gpkg')
layers = fiona.listlayers('/mnt/c/Users/earmmu/Downloads/opgrsp_gpkg_gb/Data/opgrsp_gb.gpkg')

print(layers)

# read in layers (whole of UK)
# os_gs_access =gpd.read_file('/Users/fran/Downloads/opgrsp_gpkg_gb/Data/opgrsp_gb.gpkg', layer='access_point')
os_gs_access =gpd.read_file('/mnt/c/Users/earmmu/Downloads/opgrsp_gpkg_gb/Data/opgrsp_gb.gpkg', layer='access_point')
# os_gs_site =gpd.read_file('/Users/fran/Downloads/opgrsp_gpkg_gb/Data/opgrsp_gb.gpkg', layer='greenspace_site')
os_gs_site =gpd.read_file('/mnt/c/Users/earmmu/Downloads/opgrsp_gpkg_gb/Data/opgrsp_gb.gpkg', layer='greenspace_site')

['access_point', 'greenspace_site']


In [5]:
# get boundingbox for osm highways network 
bradford_osm_boundary =osm.dissolve().bounds
bradford_osm_boundary

NameError: name 'osm' is not defined

In [ ]:
# limit bradford GS access df to the same extenet as the OSM highway data
os_gs_access_bradford = os_gs_access.cx[bradford_osm_boundary['minx']:bradford_osm_boundary['maxx'], bradford_osm_boundary['miny']:bradford_osm_boundary['maxy']]

# limit bradford GS site (polygons) df to the same extenet as the OSM highway data
os_gs_site_bradford = os_gs_site.cx[bradford_osm_boundary['minx']:bradford_osm_boundary['maxx'], bradford_osm_boundary['miny']:bradford_osm_boundary['maxy']]

In [ ]:
# have a look at greenspace access points data
os_gs_access_bradford.explore()

In [ ]:
# have a look at greenspace area data
os_gs_site_bradford.explore()

In [ ]:
# isolate peel park form the greenspace data (polygon)
peel_park =os_gs_site_bradford.loc[os_gs_site_bradford['distinctive_name_1']=='Peel Park',:]
peel_park

In [ ]:
# get all greenspaces that intersect peel park (e.g surrounding playingfields etc)
intersecting_peel_park = gpd.sjoin(os_gs_site_bradford, peel_park, how="inner", predicate="intersects")
# also define a single polygon that captures all GS in/arround peel park 
peel_park_int =intersecting_peel_park.dissolve()

In [ ]:
# have a look at the resulting peel park GS polygon
peel_park_int.explore()

In [ ]:
# get the Greenspace ids for the greenspaces intersecting peel park
gs_ids_for_peel_park =intersecting_peel_park['id_left'].unique()

# use these IDs to: 
# get access points where the reference gs site is a park that is in or intersects peel park 
peel_park_access =os_gs_access_bradford.loc[os_gs_access_bradford['ref_to_greenspace_site'].isin(gs_ids_for_peel_park),:]

In [ ]:
# drop 'index_right' column as prevents join below
peel_park_int =peel_park_int.drop(columns='index_right')
# get osm network for peel park
osm_peel_park =gpd.sjoin(osm, peel_park_int,predicate='intersects')
osm_peel_park.plot('highway', cmap='Pastel1', legend=True)

In [ ]:
fig,ax=plt.subplots(1,1, figsize=(10,10))
peel_park_int.plot(ax=ax)
peel_park_access.plot(ax=ax, color='red')
intersecting_peel_park.plot(ax=ax, alpha=0.2, color='lightblue')
osm_peel_park.plot( 'highway', cmap='Pastel1', legend=True,ax=ax)

In [ ]:
# Replace this with the path to your DEM file
# Load the DEM data
dem_data = rasterio.open('/Users/fran/Downloads/lidar_composite_dtm-2022-1-SE13se/SE13se_DTM_1m.tif')
dem_array = dem_data.read(1)
transform = dem_data.transform

 
# Plot DEM data
plt.imshow(dem_array, cmap='gray')
plt.colorbar()
plt.title('Digital Elevation Model')
plt.show()

In [ ]:
# peel_park_raster =rasterio.open('/Users/fran/Downloads/lidar_composite_dtm-2022-1-SE13se/SE13se_DTM_1m.tif', window=peel_park_int.bounds)

In [ ]:

# # Demo code that reads in a single raster and plots it
# def inverse_crop_raster_to_gdf(raster_path, gdf, output_path):
#     # Open the raster file
#     with rasterio.open(raster_path) as src:
#         # Inverse mask the raster with the GeoDataFrame's geometry
#         out_image, out_transform = mask(src, gdf.geometry, crop=True)
        
#         # Identify non-zero (non-masked) pixels
#         valid_mask = out_image[0] != 0
#         rows, cols = np.where(valid_mask)
#         min_row, max_row = rows.min(), rows.max()
#         min_col, max_col = cols.min(), cols.max()

#         # Crop to the valid data extent
#         cropped_image = out_image[:, min_row:max_row+1, min_col:max_col+1]

#         # Calculate new transform
#         from rasterio.windows import Window, transform as window_transform
#         window = Window(min_col, min_row, max_col - min_col + 1, max_row - min_row + 1)
#         cropped_transform = window_transform(window, out_transform)

        
        
#         # Update the metadata
#         out_meta = src.meta.copy()
#         out_meta.update({
#             "driver": "GTiff",
#             "height": out_image.shape[1],
#             "width": out_image.shape[2],
#         "transform": out_transform
#         })
    
#         # Write the inverse cropped raster to a new file
#         with rasterio.open(output_path, "w", **out_meta) as dest:
#             dest.write(out_image)

# inverse_crop_raster_to_gdf(raster_path='/Users/fran/Downloads/lidar_composite_dtm-2022-1-SE13se/SE13se_DTM_1m.tif', 
#                    gdf=peel_park_int, 
#                    output_path='Peel_park.tif')

# def plot_cropped_raster(raster_path):
#     with rasterio.open(raster_path) as src:
#         fig, ax = plt.subplots(figsize=(10, 10))
#         show(src, ax=ax, title="Cropped Raster")
#         plt.xlabel("Longitude")
#         plt.ylabel("Latitude")
#         plt.colorbar(ax.images[0], ax=ax, label="Elevation (m)")
#         plt.show()
# plot_cropped_raster('Peel_park.tif')

In [ ]:

# with rasterio.open('Peel_park.tif') as src:
#     print("Metadata:")
#     print(src.meta)

#     print("\nBand Information:")
#     for i in range(1, src.count + 1):
#         band = src.read(i)
#         print(f"Band {i}:")
#         print(f"  Min: {band.min()}")
#         print(f"  Max: {band.max()}")


In [ ]:

# # define funciton that assess whether each raster pixle is visible from the entrance point
 
 
# def is_visible(peel_park_raster_array, x1, y1, x2, y2):
#     """Determine if the point (x2, y2) is visible from (x1, y1) in peel_park_raster_array."""
 
#     # Calculate the number of steps for iteration based on the greater of dx or dy
#     dx = x2 - x1
#     dy = y2 - y1
#     steps = max(abs(dx), abs(dy))
 
#     # Handle the case when steps is zero
#     if steps == 0:
#         return False
 
#     # Calculate the increments for x and y
#     x_inc = dx / steps
#     y_inc = dy / steps
 
#     # Start from the source point
#     x = x1
#     y = y1
 
#     # Height at the source point
#     z = peel_park_raster_array[int(y1), int(x1)] + 20
 
#     # Iterate over the cells between source and target
#     for i in range(1, int(steps)):
#         x += x_inc
#         y += y_inc
 
#         # Calculate the elevation of the direct line at this point
#         line_elevation = z + (peel_park_raster_array[int(y2), int(x2)] - z) * (i / steps)
 
#         # If the DEM elevation at this point is higher than the direct line, it's not visible
#         if peel_park_raster_array[int(y), int(x)] > line_elevation:
#             return False
 
#     return True

# with rasterio.open('Peel_park.tif') as src:
#     peel_park_raster_array = src.read(1)
#     transform = src.transform
#     meta = src.meta
 

# # Define transmitter coordinates
# #transmitter_x, transmitter_y = 76.611022, 31.514496  # Example coordinates
# transmitter_x, transmitter_y =  peel_park_access['geometry'].iloc[0].x, peel_park_access['geometry'].iloc[0].y

# # Transforming lat long into grid coordinates
# inv_transform = ~transform
# transmitter_x, transmitter_y = [
#     int(round(coord)) for coord in inv_transform * (transmitter_x, transmitter_y)]
 
# # Create an empty array for visibility
# visibility_array = np.zeros_like(peel_park_raster_array)
 
# # Check visibility for each pixel
# for x in range(peel_park_raster_array.shape[1]):
    
#     for y in range(peel_park_raster_array.shape[0]):
#         print(x,y)
#         visibility_array[y, x] = is_visible(
#             peel_park_raster_array, transmitter_x, transmitter_y, x, y)

# # output_meta = meta.copy()
# # output_meta.update({ "dtype": "uint8" })

# # with rasterio.open('visibility_raster.tif', 'w', **output_meta) as dst:
# #     dst.write(visibility_array.astype('uint8'), 1)
# output_meta = meta.copy()
# output_meta.update({
#     "dtype": "uint8",
#     "nodata": 255 # Valid nodata value for uint8
# })

# with rasterio.open('visibility_raster.tif', 'w', **output_meta) as dst:
#     dst.write(visibility_array.astype('uint8'), 1)

 
# # with rasterio.open('data/result/los.tif', 'w', **dem_data.meta) as dst:
# #     dst.write(visibility_array, 1)
 
# plt.imshow(visibility_array, cmap='gray')
# plt.colorbar()
# plt.title('Line of Sight Analysis')
# plt.show()

In [ ]:
# define funciton that assess whether each raster pixle is visible from each entrance point listed in a pandas df
# and saves as an individual TIF for each entrance point

def is_visible_gdf(peel_park_raster_array, x1, y1, x2, y2):
    """Determine if the point (x2, y2) is visible from (x1, y1) in peel_park_raster_array."""
 
    # Calculate the number of steps for iteration based on the greater of dx or dy
    dx = x2 - x1
    dy = y2 - y1
    steps = max(abs(dx), abs(dy))
 
    # Handle the case when steps is zero
    if steps == 0:
        return False
 
    # Calculate the increments for x and y
    x_inc = dx / steps
    y_inc = dy / steps
 
    # Start from the source point
    x = x1
    y = y1
 
    # Height at the source point
    z = peel_park_raster_array[int(y1), int(x1)] + 20
 
    # Iterate over the cells between source and target
    for i in range(1, int(steps)):
        x += x_inc
        y += y_inc
 
        # Calculate the elevation of the direct line at this point
        line_elevation = z + (peel_park_raster_array[int(y2), int(x2)] - z) * (i / steps)
 
        # If the DEM elevation at this point is higher than the direct line, it's not visible
        if peel_park_raster_array[int(y), int(x)] > line_elevation:
            return False
 
    return True
 
with rasterio.open('Peel_park.tif') as src:
    peel_park_raster_array = src.read(1)
    transform = src.transform
    meta = src.meta

# Define transmitter coordinates
#transmitter_x, transmitter_y = 76.611022, 31.514496  # Example coordinates
for i in range (peel_park_access.shape[0]):
    transmitter_x, transmitter_y =  peel_park_access['geometry'].iloc[i].x, peel_park_access['geometry'].iloc[i].y
    # Transforming lat long into grid coordinates
    inv_transform = ~transform
    transmitter_x, transmitter_y = [
        int(round(coord)) for coord in inv_transform * (transmitter_x, transmitter_y)]
    
    # Create an empty array for visibility
    visibility_array = np.zeros_like(peel_park_raster_array)
    
    # Check visibility for each pixel
    for x in range(peel_park_raster_array.shape[1]):
        
        for y in range(peel_park_raster_array.shape[0]):
            print(x,y)
            visibility_array[y, x] = is_visible(
                peel_park_raster_array, transmitter_x, transmitter_y, x, y)

    # output_meta = meta.copy()
    # output_meta.update({ "dtype": "uint8" })

    # with rasterio.open('visibility_raster.tif', 'w', **output_meta) as dst:
    #     dst.write(visibility_array.astype('uint8'), 1)
    output_meta = meta.copy()
    output_meta.update({
        "dtype": "uint8",
        "nodata": 255 # Valid nodata value for uint8
    })

    with rasterio.open(f'peel_park_visability/peel_parkvisibility_raster_{i}.tif', 'w', **output_meta) as dst:
        dst.write(visibility_array.astype('uint8'), 1)

    

    

https://rasterio.readthedocs.io/en/stable/api/rasterio.merge.html combine raster layers

In [ ]:
def inverse_crop_raster_to_gdf(raster_path, gdf, output_path):
    # Open the raster file
    with rasterio.open(raster_path) as src:
        # Inverse mask the raster with the GeoDataFrame's geometry
        out_image, out_transform = mask(src, gdf.geometry, crop=True)
        
        # Identify non-zero (non-masked) pixels
        valid_mask = out_image[0] != 0
        rows, cols = np.where(valid_mask)
        min_row, max_row = rows.min(), rows.max()
        min_col, max_col = cols.min(), cols.max()

        # Crop to the valid data extent
        cropped_image = out_image[:, min_row:max_row+1, min_col:max_col+1]

        # Calculate new transform
        from rasterio.windows import Window, transform as window_transform
        window = Window(min_col, min_row, max_col - min_col + 1, max_row - min_row + 1)
        cropped_transform = window_transform(window, out_transform)

        
        
        # Update the metadata
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
        "transform": out_transform
        })
    
        # Write the inverse cropped raster to a new file
        with rasterio.open(output_path, "w", **out_meta) as dest:
            dest.write(out_image)

inverse_crop_raster_to_gdf(raster_path='peel_park_visability/peel_parkvisibility_raster_24.tif', 
                   gdf=peel_park_int, 
                   output_path='peel_park_visability/peel_parkvisibility_raster_24_cropped.tif')

In [ ]:
/Users/fran/Documents/GitHub/Park_Safety/peel_park_visability

In [ ]:
import os
import rasterio
from rasterio.mask import mask
import numpy as np
from rasterio.windows import Window, transform as window_transform

def inverse_crop_raster_to_gdf(raster_path, gdf, output_path):
    with rasterio.open(raster_path) as src:
        out_image, out_transform = mask(src, gdf.geometry, crop=True)
        
        valid_mask = out_image[0] != 0
        rows, cols = np.where(valid_mask)
        min_row, max_row = rows.min(), rows.max()
        min_col, max_col = cols.min(), cols.max()

        cropped_image = out_image[:, min_row:max_row+1, min_col:max_col+1]

        window = Window(min_col, min_row, max_col - min_col + 1, max_row - min_row + 1)
        cropped_transform = window_transform(window, out_transform)

        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": cropped_image.shape[1],
            "width": cropped_image.shape[2],
            "transform": cropped_transform
        })

        with rasterio.open(output_path, "w", **out_meta) as dest:
            dest.write(cropped_image)
        print(raster_path, ' done')

def process_directory(directory, gdf):
    for filename in os.listdir(directory):
        if filename.endswith(".tif"):
            raster_path = os.path.join(directory, filename)
            output_path = os.path.join(directory, f"cropped_{filename}")
            inverse_crop_raster_to_gdf(raster_path, gdf, output_path)
            print(f"Processed {filename} and saved as {output_path}")


In [ ]:
process_directory(directory='peel_park_visability/', 
                   gdf=peel_park_int)

In [ ]:

def merge_rasters(input_folder, output_file):
    src_files_to_mosaic = []

    for file in os.listdir(input_folder):
        if file.startswith("cropped") and file.endswith(".tif"):
        #  if file.endswith('.tif'):
            src = rasterio.open(os.path.join(input_folder, file))
            src_files_to_mosaic.append(src)

    mosaic, out_trans = merge(src_files_to_mosaic, method=copy_count)

    out_meta = src_files_to_mosaic[0].meta.copy()

    out_meta.update({
            "driver": "GTiff",
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": out_trans
    })

    with rasterio.open(output_file, "w", **out_meta) as dest:
        dest.write(mosaic)



merge_rasters('/Users/fran/Documents/GitHub/Park_Safety/peel_park_visability', 'peel_park_visability_from_entrances.tif')

In [ ]:
def plot_raster(raster_path, title):
    with rasterio.open(raster_path) as src:
        fig, ax = plt.subplots(figsize=(10, 10))
        show(src, ax=ax, title=title)
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
        plt.colorbar(ax.images[0], ax=ax, label="leg")
        plt.show()
plot_raster('peel_park_visability_from_entrances.tif', 'Park visability from entrances (based on elevation)')

Next steps:
- include vegetation and buildings in analysis
- convert rater to hex


In [ ]:
def plot_cropped_raster(raster_path):
    with rasterio.open(raster_path) as src:
        fig, ax = plt.subplots(figsize=(10, 10))
        show(src, ax=ax, title="Cropped Raster")
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
        plt.colorbar(ax.images[0], ax=ax, label="Elevation (m)")
        plt.show()

In [ ]:
plot_cropped_raster('peel_park_visability/cropped_peel_parkvisibility_raster_18.tif')

In [ ]:
plot_cropped_raster('peel_park_visability/cropped_peel_parkvisibility_raster_17.tif')

In [ ]:
plot_cropped_raster('peel_park_visability/cropped_peel_parkvisibility_raster_16.tif')

In [ ]:
plot_cropped_raster('peel_park_visability/cropped_peel_parkvisibility_raster_1.tif')

In [ ]:
plot_cropped_raster('peel_park_visability/cropped_peel_parkvisibility_raster_24.tif')